# Fast Personalized Pagerank Implementation (Moler PageRank)
I needed a fast PageRank for Wikisim project, it has to be fast enough that can run in real time on relatively small graphs. I started from optimizing the networkx, however, I found a very nice algorithm by **Cleve Mole** which takes the full advantage of sparse matrix operations. 
I implemented two versions of the algorithm in Python, both inspired  by the sparse fast solutions given in [**Cleve Moler**](https://en.wikipedia.org/wiki/Cleve_Moler)'s book, [*Experiments with MATLAB*](http://www.mathworks.com/moler/index_ncm.html). The power method is much faster with enough precision for our task. Our benchmarsk shows that this implementation is **faster than both Networkx and iGraph** implementationa magnititude of times.

## Personalized Pagerank
I modified the algorithm a little bit to be able to calculate **personalized Pagerank** as well. 

## Input Format
The input is a 2d array, each row of the array is an edge of the graph $[[a,b], [c,d]]$, $a$ and $b$ are the node numbers. The **personalization vector** is probability distribution over the nodes, a.k.a **teleporting vector**.

## Comparison with Popular Python Implementations: Networkx and iGraph
Both of the implementation (Exact Solution and PowerMethod) are much faster than their correspondent method in NetworkX. 

#

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
%%writefile ../src/pagerank.py
"""Two "fast" implementations of PageRank.

Pythom implementations of Matlab original in:
Cleve Moler, Experiments with MATLAB.
"""
# uncomment
from __future__ import division

import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"


def moler_pagerank(G, p=damping_factor,
                          personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph

    Inputs:
    -------

    G: a csr graph.
    p: damping factor
    personlize: if not None, should be an array with the size of the nodes
                containing probability distributions.
                It will be normalized automatically
    reverse: If true, returns the reversed-pagerank

    outputs
    -------

    Pagerank Scores for the nodes

    """
    # In Moler's algorithm, $G_{ij}$ represents the existences of an edge
    # from node $j$ to $i$, while we have assumed the opposite!
    if not reverse:
        G = G.T

    n, _ = G.shape
    c = sp.asarray(G.sum(axis=0)).reshape(-1)

    k = c.nonzero()[0]

    D = sprs.csr_matrix((1/c[k], (k, k)), shape=(n, n))

    if personalize is None:
        personalize = sp.ones(n)
    personalize = personalize.reshape(n, 1)
    e = (personalize/personalize.sum())*n

    I = sprs.eye(n)
    x = sprs.linalg.spsolve((I - p*G.dot(D)), e)

    x = x / x.sum()
    return x


def moler_pagerank_power(G, p=damping_factor, max_iter=100,
                                tol=1e-06, personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph

    Inputs:
    -------
    G: a csr graph.
    p: damping factor
    max_iter: maximum number of iterations
    personlize: if not None, should be an array with the size of the nodes
                containing probability distributions.
                It will be normalized automatically.
    reverse: If true, returns the reversed-pagerank

    Returns:
    --------
    Pagerank Scores for the nodes

    """
    # In Moler's algorithm, $G_{ij}$ represents the existences of an edge
    # from node $j$ to $i$, while we have assumed the opposite!
    if not reverse:
        G = G.T

    n, _ = G.shape
    c = sp.asarray(G.sum(axis=0)).reshape(-1)

    k = c.nonzero()[0]

    D = sprs.csr_matrix((1/c[k], (k, k)), shape=(n, n))

    if personalize is None:
        personalize = sp.ones(n)
    personalize = personalize.reshape(n, 1)
    e = (personalize / personalize.sum())*n

    z = (((1-p) * (c != 0) + (c == 0)) / n)[sp.newaxis, :]
    G = p * G.dot(D)

    x = e / n
    oldx = sp.zeros((n, 1))

    iteration = 0

    while sp.linalg.norm(x-oldx) > tol:
        oldx = x
        x = G.dot(x) + e.dot(z.dot(x))
        iteration += 1
        if iteration >= max_iter:
            break
    x = x / sum(x)

    return x.reshape(-1)


# Testing the algorithm

In [ ]:
!pip install python-igraph

In [ ]:
import networkx as nx
nxG = gnp_random_graph(n, , seed=None, directed=False)

In [26]:
import scipy as sp
import networkx as nx

#sp.set_printoptions(precision=4)
n=0
m=0
nxG = nx.gnm_random_graph(n=n, m=m, directed=True)
for e in nxG.edges():
     nxG[e[0]][e[1]]['weight']=sp.rand()
customization_vector = sp.random.random(n)   
customization_dict = dict(enumerate(customization_vector.reshape(-1)))
A=nx.to_scipy_sparse_matrix(nxG)        
alpha = (0.5) * sp.random.random_sample() + 0.5
pr = nx.pagerank_numpy(nxG, alpha=alpha, personalization = customization_dict) 
row, col = A.nonzero()
print("n=", n)
print("rows =", row)
print("col =", col)
print("data =", sp.array2string(A.data, precision=4, separator=','))
print ("alpha = %.2f" % (alpha, ))
# print ("custom_vector =", sp.array2string(customization_vector, precision=4, separator=','))
# print ("pagerank =", sp.array2string(sp.array(list(pr.values())), precision=4, separator=','))


NetworkXError: Graph has no nodes or edges

In [ ]:
n = 5
rows = [0 0 1 1 1 2 2 3 4 4]
col = [1 3 0 2 4 3 4 1 0 1]
data = [0.816 ,0.0907,0.8684,0.4435,0.2637,0.6541,0.8442,0.6107,0.06  ,0.3243]
alpha = 0.51
A = sp.csr()

In [9]:
n = 5
rows = [0 0 1 1 1 2 2 3 4 4]
col = [1 3 0 2 4 3 4 1 0 1]
data = [0.816 ,0.0907,0.8684,0.4435,0.2637,0.6541,0.8442,0.6107,0.06  ,0.3243]
alpha = 0.51



n = 10
rows = [1 1 1 2 3 3 4 4 4 5 6 6 7 7 7 8 8 9 9 9]
col = [4 6 8 5 4 9 6 7 8 8 2 7 0 2 9 0 7 1 3 5]
data = [0.2839,0.2941,0.9678,0.1996,0.3949,0.6571,0.2927,0.8644,0.614 ,0.9861,
 0.3228,0.5462,0.3262,0.4311,0.0764,0.1596,0.7978,0.9371,0.0692,0.4061]
alpha = 0.58


rows = [2]
col = [4]
data = [0.5441]
alpha = 0.81

n=5
rows = []
col = []
data = []
alpha = 0.70

n=0
rows = []
col = []
data = []
alpha = 0.70

array(dict_values([0.3251974433833041, 0.3357826557854313, 0.08176181929591639, 0.04711915559561643, 0.2101389259397318]),
      dtype=object)

In [ ]:
nxG.get_edge_data()

In [ ]:
# %%writefile ../test/pagerank_test.py

import networkx as nx
import random
import timeit
import numpy as np
import igraph
from numpy.testing import assert_array_almost_equal

import os
import sys
current = os.path.dirname(os.path.realpath(__file__))
src_path = os.path.join(current, '..')
sys.path.insert(0,src_path)

from src.pagerank import *

def get_random_graph(min_size=100, max_size=300, min_sparsity = 0.1, max_sparsity = 0.5):
    ''' Creates a random graph and a teleport vector and output them in different formats for different algorithms
    
    Inputs
    ------
    
    min_size and max_size: The size of the graph will be a random number in the range of (min_size, max_size)
    min_sparsity and max_sparsity: The sparcity of the graph will be a random number in the range of (min_sparsity, max_sparsity)
    
    Returns
    -------
    
    nxG: A random Graph for NetworkX
    A: The equivallent csr Adjacency matrix, for our moler_pagerank_times
    iG: The equivallent iGraph
    customization_vector: Personalization probabily vector
    customization_dict: Personalization probabily vector, in the form of a dictionary for NetworkX
    
    '''
    passed=True
    G_size = random.randint(min_size,max_size)
    p=random.uniform(min_sparsity, max_sparsity)
    nxG = nx.fast_gnp_random_graph(G_size, p, seed=None, directed=True)
    for e in nxG.edges():
         nxG[e[0]][e[1]]['weight']=sp.rand()

    A=nx.to_scipy_sparse_matrix(nxG)

    iG=igraph.Graph(list(nxG.edges()), directed=True)
    iG.es['weight'] = A.data
    
    customization_vector = np.random.random(G_size)
    customization_dict = dict(enumerate(customization_vector.reshape(-1)))
    return nxG, A, iG, customization_vector, customization_dict

import unittest

class TestMolerPageRank(unittest.TestCase):
    def setUp(self):
        self.number_of_tests = 10
    def test_exact_pagerank(self):
        damping_factor = 0.85
        for i in range(self.number_of_tests):
            nxG, A, iG, customization_vector, customization_dict = get_random_graph()

            Xnx  = nx.pagerank_numpy(nxG, alpha=damping_factor, personalization = customization_dict) 
            Xnx =  np.array([v for k,v in Xnx.items() ])

            Xml =  moler_pagerank(A, p=damping_factor, personalize=customization_vector)

            assert_array_almost_equal(Xnx,  Xml, decimal = 5)
        
    def test_power_pagerank(self):
        damping_factor = 0.85
        tol = 1e-06
        for i in range(self.number_of_tests):
            nxG, A, iG, customization_vector, customization_dict = get_random_graph()

            Ynx =  nx.pagerank_scipy(nxG, alpha=damping_factor, tol=tol, personalization=customization_dict)
            Ynx =  np.array([v for k,v in Ynx.items() ])

            Yml =  moler_pagerank_power(A, p=damping_factor, tol=tol, personalize=customization_vector)


            assert_array_almost_equal(Ynx,  Yml, decimal = 5)
            
if __name__ == '__main__':
    unittest.main()

In [ ]:
!python  ../test/pagerank_test.py

# Benchmarking

To avoid the clutter, we only visualize the fastest method from each implementation, that is: 
.

In [ ]:
%%writefile benchmarking.py
import scipy as sp
import timeit
import sys
import networkx as nx
from src.pagerank import moler_pagerank
from src.pagerank import moler_pagerank_power
from test.pagerank_test import get_random_graph

sys.path.insert(0, '..')

n = 5
number_of_graphs = 20

size_vector = sp.zeros(number_of_graphs)
netx_pagerank_times = sp.zeros(number_of_graphs)
netx_pagerank_times_numpy = sp.zeros(number_of_graphs)
netx_pagerank_times_scipy = sp.zeros(number_of_graphs)
moler_pagerank_times = sp.zeros(number_of_graphs)
moler_pagerank_times_power = sp.zeros(number_of_graphs)
ig_pagerank_times = sp.zeros(number_of_graphs)

damping_factor = 0.85
tol = 1e-3


for i in range(number_of_graphs):
    nxG, A, iG, customization_vector, customization_dict = get_random_graph(
        min_size=100, max_size=1000)
    size_vector[i] = nxG.number_of_edges()

    netx_pagerank_times[i] = timeit.timeit(
        lambda: nx.pagerank(
            nxG,
            alpha=damping_factor,
            tol=tol),
        number=n) / n
    netx_pagerank_times_numpy[i] = timeit.timeit(
        lambda: nx.pagerank_numpy(
            nxG, alpha=damping_factor), number=n) / n
    netx_pagerank_times_scipy[i] = timeit.timeit(
        lambda: nx.pagerank_scipy(
            nxG, alpha=damping_factor, tol=tol), number=n) / n

    ig_pagerank_times[i] = timeit.timeit(
        lambda: iG.personalized_pagerank(
            directed=True,
            damping=damping_factor,
            weights=iG.es['weight'],
            implementation="prpack"),
        number=n) / n

    moler_pagerank_times[i] = timeit.timeit(
        lambda: moler_pagerank(
            A, p=damping_factor), number=n) / n
    moler_pagerank_times_power[i] = timeit.timeit(
        lambda: moler_pagerank_power(
            A, p=damping_factor, tol=tol), number=n) / n


argsort = size_vector.argsort()

size_vector_sorted = size_vector[argsort]

netx_pagerank_times_sorted = netx_pagerank_times[argsort]
netx_pagerank_times_numpy_sorted = netx_pagerank_times_numpy[argsort]
netx_pagerank_times_scipy_sorted = netx_pagerank_times_scipy[argsort]

moler_pagerank_times_sorted = moler_pagerank_times[argsort]
moler_pagerank_times_power_sorted = moler_pagerank_times_power[argsort]

ig_pagerank_times_sorted = ig_pagerank_times[argsort]


print("Done")


In [ ]:
%%writefile plotting.py

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(num=None, figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')





#plt.plot(size_vector_sorted, netx_pagerank_times_sorted, 'o-',  ms=8, lw=2,alpha=0.7, color='cyan', label='networkx.PageRank')
#plt.plot(size_vector_sorted, netx_pagerank_times_numpy_sorted, 'v-', ms=8, lw=2,alpha=0.7, color='magenta', label='networkx.PageRank_numpy')
plt.plot(size_vector_sorted, netx_pagerank_times_scipy_sorted, 'P-', ms=8, lw=2,alpha=0.7, color='blue', label='networkx.PageRank_scipy')

plt.plot(size_vector_sorted, ig_pagerank_sorted, 'x-', ms=8, lw=2,alpha=0.7, color='black', label='iGraph_PageRank_ARPACK')

plt.plot(size_vector_sorted, moler_pagerank_times, '*-', ms=8, lw=2,alpha=0.7, color='red', label='moler_pagerank_times')
plt.plot(size_vector_sorted, moler_pagerank_times_power, '^-', ms=8, lw=2,alpha=0.7, color='green', label='moler_pagerank_times_Power')


plt.xlabel('Number of the edges')
plt.ylabel('Time (Seconds)')


plt.tight_layout()
plt.legend(loc=2)
plt.savefig('pagerank_exact.eps')
plt.show()


# Comparing Approximation Methods (Power Methods)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(num=None, figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')

argsort = size_vector.argsort()

size_vector_sorted = size_vector[argsort]
netx_pagerank_times_scipy_sorted = netx_pagerank_times_scipy[argsort]
moler_pagerank_times_power_sorted = moler_pagerank_times_power[argsort]



plt.plot(size_vector_sorted, netx_pagerank_times_scipy_sorted, 'P-', ms=8, lw=2,alpha=0.7, color='black', label='networkx.PageRank_scipy')
plt.plot(size_vector_sorted, moler_pagerank_times_power, '^-', ms=8, lw=2,alpha=0.7, color='green', label='moler_pagerank_times_Power')
#plt.plot(size_vector_sorted, ig_pagerank, '^-', ms=8, lw=2,alpha=0.7, color='red', label='moler_pagerank_times_Power')

plt.xlabel('Number of the edges')
plt.ylabel('Time (Seconds)')


plt.tight_layout()
plt.legend(loc=2)
plt.savefig('pagerank.eps')
plt.show()
